<a href="https://colab.research.google.com/github/AliAkbarBadri/topic-modelling/blob/master/topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [25]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
newsgroups_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
newsgroups_train['data'][0].replace('\n','')

"From: lerxst@wam.umd.edu (where's my thing)Subject: WHAT car is this!?Nntp-Posting-Host: rac3.wam.umd.eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day. It was a 2-door sports car, looked to be from the late 60s/early 70s. It was called a Bricklin. The doors were really small. In addition,the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail.Thanks,- IL   ---- brought to you by your neighborhood Lerxst ----"

# TOPIC MODELLING

## Preprocess

In [14]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [34]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
def clean_text(headline):
  le=WordNetLemmatizer()
  headline = headline.replace('\n', '')
  word_tokens = word_tokenize(headline)
  tokens = [le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text = " ".join(tokens)
  return cleaned_text

In [28]:
df = pd.DataFrame(newsgroups_train['data'], columns = ['data'])

In [35]:
df['data'] = df['data'].apply(clean_text)

In [36]:
df['data'][0]

'From lerxst wam.umd.edu thing Subject WHAT Nntp-Posting-Host rac3.wam.umd.eduOrganization University Maryland College ParkLines wondering anyone could enlighten sawthe 2-door sport looked late 60s/early called Bricklin door really small addition front bumper separate rest body This know anyone tellme model name engine spec yearsof production made history whatever info youhave funky looking please e-mail.Thanks brought neighborhood Lerxst'

In [37]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000)

In [39]:
vect_text=vect.fit_transform(df['data'])

## Latent Semantic Analysis (LSA)

In [40]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [44]:
# most important words for each topic
vocab = vect.get_feature_names_out()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
edu com would writes article subject posting like university host 

Topic 1: 
edu file windows university thanks card host posting window drive 

Topic 2: 
edu cs university game team cc pitt player uiuc year 

Topic 3: 
com article writes edu netcom hp posting host pitt inc 

Topic 4: 
game team ca player year hockey play season toronto league 

Topic 5: 
nasa gov space research access digex center moon orbit station 

Topic 6: 
ac uk cs file co window ca pitt gordon banks 

Topic 7: 
pitt cs gordon banks computer chip pittsburgh univ clipper soon 

Topic 8: 
chip clipper encryption key government game escrow state netcom phone 

Topic 9: 
ac uk drive scsi chip clipper co hard encryption key 



## Latent Dirichlet Allocation (LDA)